<a href="https://colab.research.google.com/github/pascalisnala/skripsi-machinelearning/blob/master/skripsi_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import datetime
import time
import random

import numpy as np
import pandas as pd

from collections import deque

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

from sklearn.metrics import classification_report,f1_score,accuracy_score

import random

In [37]:

df = pd.read_csv("https://raw.githubusercontent.com/pascalisnala/skripsi-machinelearning/master/dataset/data_pascal2.csv")
df.sample(2)

,timestamp,heart_rate,alpha_high,alpha_low,beta_high,beta_low,gamma_low,delta,theta,left_eye_open_prob,right_eye_open_prob,is_drowsy
3247,1586274987181,63.0,8416.0,5170.0,3344.0,4814.0,9801.0,282728.0,13794.0,0.995686,0.993952,0
6992,1586275625871,73.0,15694.0,10212.0,13431.0,6264.0,12485.0,498408.0,136965.0,0.004563,0.010437,0


In [38]:
df2 = pd.read_csv("https://raw.githubusercontent.com/pascalisnala/skripsi-machinelearning/master/dataset/data_yohannes.csv")
df2.sample(2)

,timestamp,heart_rate,alpha_high,alpha_low,beta_high,beta_low,gamma_low,delta,theta,left_eye_open_prob,right_eye_open_prob,is_drowsy
6922,1585469166120,80.0,12242.0,16767324.0,12399.0,19654.0,3709.0,836410.0,16772360.0,0.539412,0.025023,0
16264,1585470676945,84.0,1846.0,6679.0,6635.0,7533.0,2372.0,417436.0,16769971.0,0.995686,0.998022,1


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26141 entries, 0 to 26140
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   timestamp            26141 non-null  int64  
 1   heart_rate           25506 non-null  float64
 2   alpha_high           25451 non-null  float64
 3   alpha_low            25451 non-null  float64
 4   beta_high            25451 non-null  float64
 5   beta_low             25451 non-null  float64
 6   gamma_low            25451 non-null  float64
 7   delta                25451 non-null  float64
 8   theta                25451 non-null  float64
 9   left_eye_open_prob   26141 non-null  float64
 10  right_eye_open_prob  26141 non-null  float64
 11  is_drowsy            26141 non-null  int64  
dtypes: float64(10), int64(2)
memory usage: 2.4 MB


In [40]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28529 entries, 0 to 28528
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   timestamp            28529 non-null  int64  
 1   heart_rate           28435 non-null  float64
 2   alpha_high           28483 non-null  float64
 3   alpha_low            28483 non-null  float64
 4   beta_high            28483 non-null  float64
 5   beta_low             28483 non-null  float64
 6   gamma_low            28483 non-null  float64
 7   delta                28483 non-null  float64
 8   theta                28483 non-null  float64
 9   left_eye_open_prob   28529 non-null  float64
 10  right_eye_open_prob  28529 non-null  float64
 11  is_drowsy            28529 non-null  int64  
dtypes: float64(10), int64(2)
memory usage: 2.6 MB


In [41]:
df.dropna(inplace=True)
df.sample(2)

,timestamp,heart_rate,alpha_high,alpha_low,beta_high,beta_low,gamma_low,delta,theta,left_eye_open_prob,right_eye_open_prob,is_drowsy
5998,1586275459006,63.0,4029.0,5561.0,3919.0,2305.0,1840.0,252195.0,27800.0,0.993816,0.995583,0
25058,1586279068932,68.0,32212.0,31605.0,30044.0,4613.0,9613.0,92682.0,22331.0,0.991369,0.993275,1


In [42]:
df2.dropna(inplace=True)
df2.sample(2)

,timestamp,heart_rate,alpha_high,alpha_low,beta_high,beta_low,gamma_low,delta,theta,left_eye_open_prob,right_eye_open_prob,is_drowsy
6259,1585469059304,83.0,23567.0,16750093.0,18391.0,11593.0,31895.0,2300886.0,468115.0,0.985082,0.995264,0
19440,1585471193447,77.0,3936.0,16745118.0,4405.0,8062.0,1267.0,958823.0,16760030.0,0.889815,0.996212,1


In [0]:
def valid_value_only(df):
    df = df[df["heart_rate"]>=0]
    df = df[df["alpha_high"]<=15770057.0]
    df = df[df["alpha_low"]<=15770057.0]
    df = df[df["beta_high"]<=15770057.0]
    df = df[df["beta_low"]<=15770057.0]
    df = df[df["gamma_low"]<=15770057.0]
    df = df[df["delta"]<=15770057.0]
    df = df[df["theta"]<=15770057.0]
    return df
    
def data_preprocessing(df):
    df.dropna(inplace=True)
    df = valid_value_only(df)
    df['timestamp'] = pd.to_datetime(df['timestamp']/1000,unit='s')
    df.set_index("timestamp",inplace=True)
    for x in df.columns:
        if (x!="left_eye_open_prob" and x!="right_eye_open_prob" and x!="target" and x!="is_drowsy"):
            # df[x] = df[x].pct_change()
            df.replace([np.inf, -np.inf], np.nan, inplace=True)
            df.dropna(inplace=True)
            # df[x] = preprocessing.scale(df[x].values)
    return df

In [0]:
def create_data(df,seq_len,future_period):
    df["target"] = df["is_drowsy"].shift(-future_period)
    df.dropna(inplace=True)
    df["target"] = df["target"].astype('Int64')
    df = df[["heart_rate","alpha_high","left_eye_open_prob","right_eye_open_prob","target"]]
    sequential_data = []
    prev_state = deque(maxlen=seq_len)
    for i in df.values:
        prev_state.append([n for n in i[:-1]])
        if len(prev_state)==seq_len:
            sequential_data.append([np.array(prev_state), i[-1]])
    return df, sequential_data

In [45]:
df = data_preprocessing(df)
df.sample(2)

,heart_rate,alpha_high,alpha_low,beta_high,beta_low,gamma_low,delta,theta,left_eye_open_prob,right_eye_open_prob,is_drowsy
timestamp,,,,,,,,,,,
2020-04-07 16:31:41.551000118,59.0,68170.0,17826.0,32571.0,26991.0,29434.0,1653532.0,285609.0,0.945900,0.683255,0
2020-04-07 16:57:17.065999985,79.0,988.0,9695.0,3305.0,3396.0,6149.0,143428.0,12896.0,0.993816,0.990413,1


In [46]:
df = data_preprocessing(df2)
df2.sample(2)

,timestamp,heart_rate,alpha_high,alpha_low,beta_high,beta_low,gamma_low,delta,theta,left_eye_open_prob,right_eye_open_prob,is_drowsy
14788,1585470437579,95.0,2343.0,3183.0,2439.0,3458.0,782.0,156078.0,10172.0,0.038779,0.872311,1
17388,1585470860561,100.0,5756.0,17651.0,4407.0,938.0,2161.0,144595.0,16764853.0,0.984551,0.993616,1


In [47]:
SEQ_LEN = 3000
FUTURE_PERIOD_PREDICT = 1500
df, sequential_data = create_data(df,SEQ_LEN,FUTURE_PERIOD_PREDICT)
df.sample(2)

,heart_rate,alpha_high,left_eye_open_prob,right_eye_open_prob,target
timestamp,,,,,
2020-03-29 08:33:42.473999977,93.0,1737.0,0.986606,0.994941,1
2020-03-29 08:28:53.759999990,84.0,14246.0,0.951442,0.986117,1


In [54]:
df.sample(2)

,heart_rate,alpha_high,left_eye_open_prob,right_eye_open_prob,target
timestamp,,,,,
2020-03-29 08:42:34.128000020,81.0,4709.0,0.995686,0.987686,1
2020-03-29 07:55:46.878999949,84.0,77657.0,0.804988,0.996827,0


In [55]:
df2, sequential_data2 = create_data(df2,SEQ_LEN,FUTURE_PERIOD_PREDICT)
df2.sample(2)

,heart_rate,alpha_high,left_eye_open_prob,right_eye_open_prob,target
12376,89.0,71065.0,0.972051,0.983414,1
17118,90.0,13630.0,0.991869,0.992231,1


In [0]:
sequential_data = sequential_data + sequential_data2

In [0]:
drowsy=[]
alert=[]
for seq, target in sequential_data:
    if target==0:
        drowsy.append([seq,target])
    else:
        alert.append([seq,target])

random.shuffle(drowsy)
random.shuffle(alert)

lower = min(len(drowsy), len(alert))

drowsy = drowsy[:lower]
alert = alert[:lower]

sequential_data = drowsy + alert
random.shuffle(sequential_data)

In [0]:
def create_training_label_data(sequential_data): 
    X = []
    y =[]
    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)
    return np.array(X),np.array(y)

In [0]:
X,y = create_training_label_data(sequential_data)

In [62]:
EPOCHS = 3
BATCH_SIZE = 64
NAME = f"LSTM-SKRIPSI-{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

model = Sequential()
model.add(LSTM(128, input_shape=X.shape[1:], return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape=X.shape[1:], return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape=X.shape[1:]))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(1, activation="sigmoid"))

opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(loss="binary_crossentropy",
             optimizer=opt,
             metrics=["accuracy"])

tensorboard = TensorBoard(log_dir=f'logs\\{NAME}')
filepath = "LSTM_Skripsi-{epoch:02d}-{val_acc:.3f}"  # unique file name that will include the epoch and the validation acc for that epoch
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')) # saves only the best ones


history = model.fit(X,
                   y,
                   batch_size=BATCH_SIZE,
                   epochs=EPOCHS,
                   verbose=1)

Epoch 1/3
303/303 [==============================] - 540s 2s/step - loss: 0.1639 - accuracy: 0.9445
Epoch 2/3
303/303 [==============================] - 539s 2s/step - loss: 0.1830 - accuracy: 0.9331
Epoch 3/3
303/303 [==============================] - 539s 2s/step - loss: 0.1864 - accuracy: 0.9376


In [63]:
model_predict = model.predict(X)
prediction = np.where(model_predict> 0.5, 1, 0)
print(classification_report(prediction,y))

              precision    recall  f1-score   support

           0       0.87      0.91      0.89      9245
           1       0.91      0.87      0.89     10131

    accuracy                           0.89     19376
   macro avg       0.89      0.89      0.89     19376
weighted avg       0.89      0.89      0.89     19376



In [64]:
print(f1_score(prediction,y))

0.8912659569100359


In [0]:
model_predict

array([[0.9611082 ],
       [0.7562449 ],
       [0.9640582 ],
       ...,
       [0.833615  ],
       [0.87276185],
       [0.9575619 ]], dtype=float32)

In [65]:
df_test = pd.read_csv("https://raw.githubusercontent.com/pascalisnala/skripsi-machinelearning/master/dataset/data_pascal3.csv")
df_test.sample(2)

,timestamp,heart_rate,alpha_high,alpha_low,beta_high,beta_low,gamma_low,delta,theta,left_eye_open_prob,right_eye_open_prob,is_drowsy
20266,1586412880559,92.0,8542.0,31612.0,11960.0,14468.0,3260.0,1552286.0,16776223.0,0.997039,0.999449,0
3035,1586410656868,79.0,2016.0,186.0,2723.0,1121.0,4298.0,59436.0,17251.0,0.991869,0.995899,0


In [66]:
df_test = data_preprocessing(df_test)
df_test.sample(2)

,heart_rate,alpha_high,alpha_low,beta_high,beta_low,gamma_low,delta,theta,left_eye_open_prob,right_eye_open_prob,is_drowsy
timestamp,,,,,,,,,,,
2020-04-09 05:55:16.855000019,88.0,21714.0,9654.0,6061.0,5339.0,1818.0,663878.0,25370.0,0.996592,0.997431,0
2020-04-09 05:56:49.848999977,85.0,12755.0,12213.0,19480.0,9923.0,6522.0,1720102.0,31682.0,0.988791,0.998313,0


In [67]:
df_test, sequential_data_test = create_data(df_test,SEQ_LEN,FUTURE_PERIOD_PREDICT)
df_test.sample(2)

,heart_rate,alpha_high,left_eye_open_prob,right_eye_open_prob,target
timestamp,,,,,
2020-04-09 06:00:27.243000031,83.0,103309.0,0.165287,0.992931,0
2020-04-09 05:40:16.082000017,82.0,344372.0,0.993816,0.997728,0


In [0]:
random.shuffle(sequential_data_test)

In [0]:
X_test, y_test = create_training_label_data(sequential_data_test)

In [70]:
model_predict = model.predict(X_test)
prediction = np.where(model_predict> 0.5, 1, 0)
print(classification_report(prediction,y_test))

              precision    recall  f1-score   support

           0       0.88      0.89      0.88     13786
           1       0.51      0.49      0.50      3348

    accuracy                           0.81     17134
   macro avg       0.70      0.69      0.69     17134
weighted avg       0.81      0.81      0.81     17134



In [82]:
accuracy_score(prediction,y_test)

0.8097350297653788

In [83]:
f1_score(prediction,y_test)

0.504107088530575

In [79]:
unique_elements, counts_elements = np.unique(y_test, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

[[    0     1]
 [13908  3226]]


In [75]:
model_predict = model.predict(X_test)
prediction = np.where(model_predict> 0.5, 1, 0)
print(classification_report(prediction,y_test))

array([[0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0]])

In [76]:
y_test[0:10]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 1])

In [84]:
model.save("model3.model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model3.model/assets


In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [86]:
open("model3.tflite", "wb").write(tflite_model)

2933312

In [87]:
tf.saved_model.save(model, "model3")

INFO:tensorflow:Assets written to: model3/assets


INFO:tensorflow:Assets written to: model3/assets


In [88]:
!zip -r /content/file.zip /content/model3

  adding: content/model3/ (stored 0%)
  adding: content/model3/variables/ (stored 0%)
  adding: content/model3/variables/variables.index (deflated 73%)
  adding: content/model3/variables/variables.data-00001-of-00002 (deflated 6%)
  adding: content/model3/variables/variables.data-00000-of-00002 (deflated 81%)
  adding: content/model3/assets/ (stored 0%)
  adding: content/model3/saved_model.pb (deflated 90%)


In [0]:
interpreter = tf.lite.Interpreter(model_path="model3.tflite")
interpreter.allocate_tensors()

In [90]:
print(interpreter.get_input_details()[0]['shape'])  
print(interpreter.get_input_details()[0]['dtype'])  

[   1 3000    4]
<class 'numpy.float32'>


In [91]:
print(interpreter.get_output_details()[0]['shape'])  
print(interpreter.get_output_details()[0]['dtype'])  

[]
<class 'numpy.float32'>
